In [15]:
import struct

MASK64 = 0xFFFFFFFFFFFFFFFF

def ascon_permutation(state, rounds=12):
    constants = [0xf0, 0xe1, 0xd2, 0xc3, 0xb4, 0xa5, 0x96, 0x87, 0x78, 0x69, 0x5a, 0x4b]
    for i in range(12 - rounds, 12):
        state[2] ^= constants[i]
        # Substitution Layer
        state[0] ^= state[4]; state[4] ^= state[3]; state[2] ^= state[1]
        t = [(~state[j] & state[(j+1)%5]) & MASK64 for j in range(5)]
        for j in range(5): state[j] ^= t[j]
        state[1] ^= state[0]; state[0] ^= state[4]; state[3] ^= state[2]; state[2] = (~state[2]) & MASK64
        # Linear Diffusion Layer
        state[0] ^= (state[0] >> 19 | state[0] << 45) ^ (state[0] >> 28 | state[0] << 36); state[0] &= MASK64
        state[1] ^= (state[1] >> 61 | state[1] << 3)  ^ (state[1] >> 39 | state[1] << 25); state[1] &= MASK64
        state[2] ^= (state[2] >> 1  | state[2] << 63) ^ (state[2] >> 6  | state[2] << 58); state[2] &= MASK64
        state[3] ^= (state[3] >> 10 | state[3] << 54) ^ (state[3] >> 17 | state[3] << 47); state[3] &= MASK64
        state[4] ^= (state[4] >> 7  | state[4] << 57) ^ (state[4] >> 41 | state[4] << 23); state[4] &= MASK64

def ascon_encrypt_full(key, nonce, plaintext):
    # Initialization
    k0, k1 = struct.unpack(">QQ", key)
    n0, n1 = struct.unpack(">QQ", nonce)
    state = [0x80400c0600000000, k0, k1, n0, n1]
    ascon_permutation(state, 12)
    state[3] ^= k0; state[4] ^= k1
    
    # Padding: Add 0x80 then 0x00s until multiple of 8
    padded_pt = plaintext + b'\x80' + b'\x00' * (7 - (len(plaintext) % 8))
    ciphertext = b""
    
    # Processing blocks (6-round permutation between blocks)
    for i in range(0, len(padded_pt), 8):
        p_block = struct.unpack(">Q", padded_pt[i:i+8])[0]
        state[0] ^= p_block
        ciphertext += struct.pack(">Q", state[0])
        if i + 8 < len(padded_pt):
            ascon_permutation(state, 6)
            
    # Finalization & Tag
    state[1] ^= k0; state[2] ^= k1
    ascon_permutation(state, 12)
    tag = struct.pack(">QQ", (state[3] ^ k0) & MASK64, (state[4] ^ k1) & MASK64)
    return ciphertext, tag

def ascon_decrypt_full(key, nonce, ciphertext, tag):
    k0, k1 = struct.unpack(">QQ", key)
    n0, n1 = struct.unpack(">QQ", nonce)
    state = [0x80400c0600000000, k0, k1, n0, n1]
    ascon_permutation(state, 12)
    state[3] ^= k0; state[4] ^= k1
    
    padded_pt = b""
    for i in range(0, len(ciphertext), 8):
        c_block = struct.unpack(">Q", ciphertext[i:i+8])[0]
        p_block = state[0] ^ c_block
        padded_pt += struct.pack(">Q", p_block)
        state[0] = c_block # Update state with ciphertext
        if i + 8 < len(ciphertext):
            ascon_permutation(state, 6)
            
    # Tag Verification
    state[1] ^= k0; state[2] ^= k1
    ascon_permutation(state, 12)
    v_tag = struct.pack(">QQ", (state[3] ^ k0) & MASK64, (state[4] ^ k1) & MASK64)
    
    if v_tag != tag: return None
    # Remove padding (strip trailing 0x00s and the last 0x80)
    return padded_pt.rstrip(b'\x00')[:-1]

# Run it
k = b"SECRET_KEY_12345"
n = b"NONCE_VALUE_9876"
msg = b"Let there be light"

print(f"Original: {msg}")
print(f"Original Length: {len(msg)} bytes\n")

ct, t = ascon_encrypt_full(k, n, msg)
result = ascon_decrypt_full(k, n, ct, t)

print(f"Ciphertext: {ct}")
print(f"Ciphertext Length: {len(ct)} bytes")
print(f"Authentication Tag: {t}")
print(f"Authentication Tag Length: {len(t)} bytes\n")

print(f"Decrypted: {result.decode()}")
print(f"Integrity: {'Verified' if msg == result else 'FAILED'}")

Original: b'Let there be light'
Original Length: 18 bytes

Ciphertext: b'^A0d\xce\x1dru\xa2\x86\x08\x17$t\x0c\xe4[\xc4\xa8\x119\x04\xa4\xb2'
Ciphertext Length: 24 bytes
Authentication Tag: b"\x0b\xb7\x8eX7\xa4\xdck';;F\xd8\x8b\x0b\x9b"
Authentication Tag Length: 16 bytes

Decrypted: Let there be light
Integrity: Verified
